# Código usado para la obtención de resultados

Importo las librerías que voy a usar.

In [ ]:
import numpy as np
from time import time

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb

Defino funciones para generar las matrices de adyacencia del $\textit{Lights Out clásico}$, estas son compatibles con tableros rectangulares de tamaño arbitrario:

In [ ]:
def accion_clasico(y_toggle, x_toggle, height, width):
    # Esta función genera una matriz que al aplanarse (función flatten)
    # coincide con la acción del botón correspondiente al índice 
    # determinado por sus coordenadas
    
    # Comienzo con una matriz nula de tamaño dado
    toggle = np.zeros((height, width))
    # El botón que se presiona sufre alternación 
    toggle[y_toggle][x_toggle] = 1
    
    # Valido condiciones de que los vecinos no diagonales estén
    # contenidos en los límites del tablero, si se cumplen
    # es porque el botón sufre alternación
    if (y_toggle - 1 >= 0):          # Botón superior
        toggle[y_toggle - 1][x_toggle] = 1
    if (y_toggle + 1 <= height - 1): # Botón inferior
        toggle[y_toggle + 1][x_toggle] = 1
    if (x_toggle - 1 >= 0):          # Botón izquierdo
        toggle[y_toggle][x_toggle - 1] = 1
    if (x_toggle + 1 <= width - 1):  # Botón derecho
        toggle[y_toggle][x_toggle + 1] = 1
    
    return toggle # Con la matriz construida hago retorno

def construccion_clasico(height, width):
    # Esta función construye la matriz de adyacencia a partir de 
    # las acciones de los botones del tablero
    
    # Inicio creando una matriz compatible sin elementos, esto 
    # es un truco para que al agregar filas no haya problemas
    adjacency = np.zeros((0, height*width))
    
    # El proceso se hace una vez por cada botón del tablero
    for i in range(height):
        for j in range(width):
            # Construyo la acción como matriz en el orden de indexación
            toggle = accion_clasico(i, j, height, width)
            # La aplano en forma de fila como en el modelo algebraico
            row = toggle.flatten()
            # Añado la fila construida a la matriz de adyacencia
            adjacency = np.vstack((adjacency,row))
    
    # Retorno la matriz como lista porque el algoritmo que haya la
    # matriz rref fue implementado para trabajar con este tipo de dato
    return adjacency.tolist()

Defino funciones para generar las matrices de adyacencia de la $\textit{variante toroidal de Lights Out}$, estas son compatibles con tableros rectangulares de tamaño arbitrario:

In [ ]:
def accion_toroidal(y_toggle, x_toggle, height, width):
    # Esta función hace el proceso de la función 'accion_normal'
    # pero para la variante toroidal, por lo que su lógica es
    # casi idéntica
    toggle = np.zeros((height, width))
    
    toggle[y_toggle][x_toggle] = 1
    
    # Sólo cambia en que cuando un vecino no diagonal escapa de
    # los límites del tablero, se marca la alternancia en el
    # borde opuesto del tablero
    if (y_toggle - 1 >= 0):
        toggle[y_toggle - 1][x_toggle] = 1
    else:
        toggle[height - 1][x_toggle] = 1
    if (y_toggle + 1 <= height - 1):
        toggle[y_toggle + 1][x_toggle] = 1
    else:
        toggle[0][x_toggle] = 1
    if (x_toggle - 1 >= 0):
        toggle[y_toggle][x_toggle - 1] = 1
    else:
        toggle[y_toggle][width - 1] = 1
    if (x_toggle + 1 <= width - 1):
        toggle[y_toggle][x_toggle + 1] = 1
    else:
        toggle[y_toggle][0] = 1
    
    return toggle

def construccion_toroidal(height, width):
    # Esta función hace el proceso de la función 'construccion_normal'
    # pero para la variante toroidal, por lo que su lógica es
    # casi idéntica
    adjacency = np.zeros((0, height*width))
    
    for i in range(height):
        for j in range(width):
            # Sólo cambia al depender de las acciones toroidales
            toggle = accion_toroidal(i, j, height, width)
            row = toggle.flatten()
            adjacency = np.vstack((adjacency,row))
    
    return adjacency.tolist()

Se definen dos funciones: una que calcula el opuesto aditivo de un número $e$ en el cuerpo $\mathbb{Z}/m\mathbb{Z}$ con $m$ dado y otra que, de forma similar, calcula su opuesto multiplicativo:

In [ ]:
def elemento_opuesto(e, base):
    # Esta función proviene de la tesis de Salamanca sin cambios;
    # busca entre los elementos del cuerpo el primer elemento de
    # cero al sumarse con 'e'. Como Z/mZ es un cuerpo, la función
    # tiene un retorno único garantizado
    for k in range(base):   
        if (k + e) % base == 0:   
            return k
        
def elemento_inverso(e, base):
    # Esta función proviene de la tesis de Salamanca sin cambios.
    # La lógica de esta función es idéntica a la de la función
    # 'elemento_opuesto'
    for k in range(base):
        # Sólo cambia al depender de la estructura multiplicativa
        if (k * e) % base == 1:
            return k 

Se define una función que halla la matriz $\textit{rref}$ de una matriz dada que contenga elementos del cuerpo $\mathbb{Z}/m\mathbb{Z}$ por medio del método de Gauss-Jordan. La función funciona apropiadamente siempre que $m$ sea primo:

In [ ]:
def rref_gauss_jordan(L, base):
    # Esta función proviene de la tesis de Salamanca, halla la
    # matriz rref por medio de reducción de Gauss-Jordan. 
    # Cambios: la lógica de reducción por filas se pasa a una 
    # función de nombre 'reduccion_filas' para mejorar la legibilidad. 
    
    # Se recorren todas las filas en orden descendente aplicando
    # operaciones de reducción por filas que dependen del valor
    # particular de los términos de la diagonal principal
    for i in range(len(L)): 
        
        # En caso de que el término que debe ser pivote valga 0.
        # Los juegos a trabajar son reflexivos, este caso no les aplica
        if L[i][i] == 0:
            # Se busca el primer candidato a pivote posible en las
            # filas inferiores. Luego se permutan las filas involucradas
            for k in range (i + 1, len(L)): 
                if L[k][i] == 1:     
                    filaAux = L[i]   
                    L[i] = L[k]   
                    L[k] = filaAux  
                    break    
            # Se anulan los otros términos de la columna sumando filas
            L = reduccion_filas(L, base, i)   
        
        # En caso de que el término que debe ser pivote ya valga 1
        elif L[i][i] == 1:  
            # Se anulan los otros términos de la columna sumando filas
            L = reduccion_filas(L, base, i)
                
        
        # Este caso sólo aplica cuando m >= 2. No se usa en este caso
        # donde m = 2
        else:  
            # Se escala la fila para que el término que debe ser
            # ser pivote valga 1
            inverso = elemento_inverso(L[i][i], base)
            for j in range(len(L[i])):    
                L[i][j] = (L[i][j]*inverso) % base      
            
            # Se anulan los otros términos de la columna sumando filas
            L = reduccion_filas(L, base, i)     

    return L

def reduccion_filas(L, base, i):
    # Esta función hace operaciones de reducción por filas sobre las
    # filas distintas a la i-ésima, anulando todos los términos de
    # la columna distintos al pivote
    for j in range(len(L)):   
        if L[j][i] != 0 and j != i:
            piv = elemento_opuesto(L[j][i], base) 
            for k in range(len(L[0])): 
                L[j][k] = (L[j][k] + piv * L[i][k]) % base
    return L

Se define una función que halla la nulidad de una matriz dada:

In [ ]:
def nulidad(L):  
    # Esta función proviene de la tesis de Salamanca sin cambios;
    # calcula la nulidad de la matriz como el número de filas
    # nulas, para esto recorre las filas e incrementa un contador
    # inicialmente nulo con cada nueva fila nula encontrada
    cont = 0        
    for i in range(len(L)):  
        todo_ceros = True     
        for j in range(len(L[0])):    
            if L[i][j] != 0:      
                todo_ceros = False    
        if todo_ceros:   
            cont += 1 
    return cont

Defino una función para reproducir los resultados del artículo base:

In [ ]:
def res_tab_cuad(a, b, txt_sucesion, txt_tiempo, func_cons):
    # Esta función obtiene los valores de nul(A_n) entre dos límites
    # (incluyéndolos), además de los tiempos de generación de las 
    # matrices de adyacencia y las rref. Consigna estos datos en dos
    # archivos de texto (.txt) distintos.
    # La apertura y cierre oportunos de los .txt mejora los tiempos
    # de ejecución
    
    archivo_suc = open(txt_sucesion + ".txt", "a")
    archivo_tiempo = open(txt_tiempo + ".txt", "a")
    
    # Anotación de límites de las filas
    archivo_suc.write("lim_inf = " + str(a) +" \n")
    archivo_suc.write("lim_sup = " + str(b) +" \n \n")
    
    archivo_tiempo.write("lim_inf = " + str(a) +" \n")
    archivo_tiempo.write("lim_sup = " + str(b) +" \n \n")
    
    # Anotación de rótulos de las columnas
    archivo_suc.write("i nul \n")
    archivo_tiempo.write("i t_matriz t_rref \n")
    
    archivo_suc.close()
    archivo_tiempo.close()
    
    for i in range(a, b + 1):
        # Matrices de adyacencia y conteo de tiempo
        tiempo_inicio = time()
        adyacencia = func_cons(i, i)
        tiempo_matriz = time() - tiempo_inicio
        
        # Matrices rref y conteo de tiempo
        tiempo_inicio = time()
        rref = rref_gauss_jordan(adyacencia, 2)
        tiempo_rref = time() - tiempo_inicio
        
        nul = nulidad(rref) # Cálculo de nul(A_n)
        
        # Escritura de resultados
        archivo_suc = open(txt_sucesion + ".txt", "a")
        archivo_tiempo = open(txt_tiempo + ".txt", "a")
        
        archivo_suc.write(str(i) + ", " + str(nul) + "\n")
        archivo_tiempo.write(str(i) + ", " + str(tiempo_matriz) + ", " + str(tiempo_rref) + "\n")
        
        archivo_suc.close()
        archivo_tiempo.close()
    
    # Espacio adicional entre procesos
    archivo_suc = open(txt_sucesion + ".txt", "a")
    archivo_tiempo = open(txt_tiempo + ".txt", "a")
    
    archivo_suc.write("\n")
    archivo_tiempo.write("\n")
    
    archivo_suc.close()
    archivo_tiempo.close()

Defino una función para obtener los resultados necesarios del problema del triángulo:

In [ ]:
def res_tab_arb(a, b, txt_mapa, func_cons):
    # Esta función obtiene los valores de la nulidad de la matriz de
    # adyacencia entre entre dos límites a <= i <= b (incluyéndolos),
    # para todos los tableros rectangulares entre ellos, de esta forma,
    # la otra dimensión j del tablero queda libre entre 1 <= j <= i.
    # Consigna los datos obtenidos en un archivo de texto.
    # La apertura y cierre oportunos de los .txt mejora los tiempos
    # de ejecución
    
    archivo_mapa = open(txt_mapa + ".txt", "a")
    
    # Anotación de límites de las filas
    archivo_mapa.write("lim_inf = " + str(a) +" \n")
    archivo_mapa.write("lim_sup = " + str(b) +" \n \n")
    # Anotación de rótulos de las columnas
    archivo_mapa.write("i j nul \n")
    
    archivo_mapa.close()
    
    for i in range(a, b + 1):
        for j in range(1, i + 1):
            # Cálculo de la nulidad entre los límites establecidos
            adyacencia = func_cons(i, j)
            rref = rref_gauss_jordan(adyacencia, 2)
            nul = nulidad(rref)
            
            # Escritura de resultados
            archivo_mapa = open(txt_mapa + ".txt", "a")
            archivo_mapa.write(str(i) + ", " + str(j) + ", " + str(nul) + "\n")
            archivo_mapa.close()
    
    # Espacio adicional entre procesos
    archivo_mapa = open(txt_mapa + ".txt", "a")
    archivo_mapa.write("\n")
    archivo_mapa.close()

A través de las siguientes lineas reproducimos los resultados computacionales del artículo base:

In [ ]:
res_tab_cuad(1, 60, "sucesion_clasico", "tiempo_clasico", construccion_clasico)

In [ ]:
res_tab_cuad(1, 60, "sucesion_toroidal", "tiempo_toroidal", construccion_toroidal)

A través de las siguientes lineas calculamos los resultados obtenidos sobre el problema del triángulo:

In [ ]:
res_tab_arb(1, 60, "mapa_clasico", construccion_clasico)

In [ ]:
res_tab_arb(1, 60, "mapa_toroidal", construccion_toroidal)

Defino dos funciones: una para obtener graficas sobre los contenidos del artículo base y otra para obtener gráficas del problema del triángulo.

Para ejecutar estas funciones se requiere copiar los archivos de resultados que se van a utilizar en una carpeta interna de nombre 'resultados' y que a estos se les quiten los encabezados. Esto se hace para tener copias intactas con los resultados con los encabezados. Los encabezados son información valiosa sobre el progreso realizado.

In [ ]:
def grafica_articulo(archivo, nombre_matriz):
    # Esta función grafica los resultados de la reproducción de resultados
    # del artículo base para los Lights Out y sus variantes toroidales
    
    # Lectura del archivo de resultados
    df = pd.read_table('resultados/' + archivo + '.txt', sep = ', ', engine = 'python', names = ('Orden', 'Valores'))
    
    # Definición del tamaño de la figura
    sb.set(rc = {'figure.figsize': (20, 15)})
    
    # Construcción y generación de la imagen
    plt.xlabel('$n$')
    plt.ylabel('$nul(' + nombre_matriz + ')$')
    plt.title('Orden $n$ vs. $nul(' + nombre_matriz + ')$')
    plt.plot(df.Orden, df.Valores)

def grafica_triangulo(archivo):
    # Esta función grafica los resultados del problema del
    # triángulo para los Lights Out y sus variantes toroidales
    
    # Lectura del archivo de resultados
    df = pd.read_table('resultados/' + archivo + '.txt', sep = ', ', engine = 'python', names = ('Filas', 'Columnas', 'Valores'))
    
    # Paso de columnas a un DataFrame
    df.drop_duplicates(['Filas', 'Columnas'], inplace = True)
    pivot = df.pivot(index = 'Filas', columns = 'Columnas', values = 'Valores')
    
    # Definición del tamaño de la figura
    sb.set(rc = {'figure.figsize': (20, 15)})
    # Construcción de la imagen
    sb.heatmap(pivot, annot = True)

A través de las siguientes lineas graficamos la reproducción de resultados del artículo base:

In [ ]:
grafica_articulo('sucesion_clasico', 'C_n')

In [ ]:
grafica_articulo('sucesion_toroidal', 'T_n')

A través de las siguientes lineas graficamos los resultados obtenidos para los problemas del triángulo:

In [ ]:
grafica_triangulo('mapa_clasico')

In [ ]:
grafica_triangulo('mapa_toroidal')